In [75]:
print(__doc__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_rows", None, "display.max_columns", None)

Automatically created module for IPython interactive environment


In [76]:
path = r'/Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/09021120_EQODS_NF(200,10)'
all_files = glob.glob(os.path.join(path, "*.csv"))

min_abs = []
idx_min_abs = []
min_diff = []
idx_min_diff = []


for f in all_files:
    df = pd.read_csv(f)
    df = df.rename(columns={"# x_axis": "axis", " ur_accuracies": "ur_accuracies", " ur_fairness": "ur_fairness", " rs_accuracies": "rs_accuracies", " rs_fairness": "rs_fairness", " nonal_accuracies": "nonal_accuracies", " nonal_fairness": "nonal_fairness"})
    min_abs = np.append(min_abs, df['ur_fairness'].min())
    # print(min_abs)
    idx_min_abs = np.append(idx_min_abs, df['ur_fairness'].idxmin())
    # print(idx_min_abs)
    df_diff = df.diff()
    min_diff = np.append(min_diff, df_diff['ur_fairness'].min())
    # print(min_diff)
    idx_min_diff = np.append(idx_min_diff, df_diff['ur_fairness'].idxmin())
    # print(idx_min_diff)

df_overview = pd.DataFrame(np.hstack((min_abs[:,None], idx_min_abs[:,None], min_diff[:,None], idx_min_diff[:,None])))
print(df_overview)

    

    
    
        


    axis  ur_accuracies  ur_fairness  rs_accuracies  rs_fairness  \
0     10        57.6294       0.0672        57.6294       0.0672   
1     20        58.4469       0.0790        54.0872       0.0263   
2     30        57.6975       0.0739        59.2643       0.0815   
3     40        57.6294       0.0731        62.0572       0.1097   
4     50        57.4932       0.0695        61.4441       0.1324   
5     60        58.9918       0.0835        63.8283       0.0883   
6     70        59.2643       0.1048        62.4659       0.1317   
7     80        59.1281       0.1077        64.0327       0.1094   
8     90        60.6267       0.1054        63.5559       0.1544   
9    100        60.0136       0.0976        63.2153       0.1498   
10   110        59.6730       0.1115        62.9428       0.1336   
11   120        60.0136       0.1139        62.4659       0.1656   
12   130        60.1499       0.1184        65.1226       0.1486   
13   140        60.1499       0.1180        66.2

In [77]:
# print(concatenated_df)
budget = []
ur_a_std = []
ur_f_std = []
rs_a_std = []
rs_f_std = []
nonal_a_std = []
nonal_f_std = []
ur_a_mean = []
ur_f_mean = []
rs_a_mean = []
rs_f_mean = []
nonal_a_mean = []
nonal_f_mean = []

# print(concatenated_df)

for i in pd.unique(concatenated_df['axis']):
    
    df_per_budget = concatenated_df[(concatenated_df['axis'] == i)].copy()
    df_per_budget

    # stdev = (df_per_budget.loc[:, df_per_budget.columns != 'axis'].std(axis = 0))
    # mean = (df_per_budget.loc[:, df_per_budget.columns != 'axis'].mean(axis = 0))

    budget = np.append(budget, i)


    # ur_a_std = np.append(ur_a_std, stdev['ur_accuracies'])
    # ur_f_std = np.append(ur_f_std, stdev['ur_fairness'])
    # rs_a_std = np.append(rs_a_std, stdev['rs_accuracies'])
    # rs_f_std = np.append(rs_f_std, stdev['rs_fairness'])
    # nonal_a_std = np.append(nonal_a_std, stdev['nonal_accuracies'])
    # nonal_f_std = np.append(nonal_f_std, stdev['nonal_fairness'])

    # ur_a_mean = np.append(ur_a_mean, mean['ur_accuracies'])
    # ur_f_mean = np.append(ur_f_mean, mean['ur_fairness'])
    # rs_a_mean = np.append(rs_a_mean, mean['rs_accuracies'])
    # rs_f_mean = np.append(rs_f_mean, mean['rs_fairness'])
    # nonal_a_mean = np.append(nonal_a_mean, mean['nonal_accuracies'])
    # nonal_f_mean = np.append(nonal_f_mean, mean['nonal_fairness'])

# fig = plt.figure(figsize=(10,5))
# plt.plot(budget, ur_a_std, label='unfair-active')
# plt.plot(budget, rs_a_std, label='random-active')
# plt.plot(budget, nonal_a_std, label='non-active')
# plt.legend()
# plt.savefig("result_log/a_std.png", bbox_inches='tight', dpi=200)
# plt.show()

# fig = plt.figure(figsize=(10,5))
# plt.plot(budget, ur_f_std, label='unfair-active')
# plt.plot(budget, rs_f_std, label='random-active')
# plt.plot(budget, nonal_f_std, label='non-active')
# plt.legend()
# plt.savefig("result_log/f_std.png", bbox_inches='tight', dpi=200)
# plt.show()

# fig = plt.figure(figsize=(10,5))
# plt.plot(budget, ur_a_mean, label='unfair-active')
# plt.plot(budget, rs_a_mean, label='random-active')
# plt.plot(budget, nonal_a_mean, label='non-active')
# plt.legend()
# plt.savefig("result_log/a_avg.png", bbox_inches='tight', dpi=200)
# plt.show()

# fig = plt.figure(figsize=(10,5))
# plt.plot(budget, ur_f_mean, label='unfair-active')
# plt.plot(budget, rs_f_mean, label='random-active')
# plt.plot(budget, nonal_f_mean, label='non-active')
# plt.legend()
# plt.savefig("result_log/f_avg.png", bbox_inches='tight', dpi=200)
# plt.show()

# ur_index = np.argsort(ur_a_mean)
# rs_index = np.argsort(rs_a_mean)
# nonal_index = np.argsort(nonal_a_mean) 

# fig = plt.figure(figsize=(10,5))
# plt.plot(ur_a_mean[ur_index], ur_f_mean[ur_index], label='unfair-active')
# plt.plot(rs_a_mean[rs_index], rs_f_mean[rs_index], label='random-active')
# plt.plot(nonal_a_mean[nonal_index], nonal_f_mean[nonal_index], label='non-active')
# plt.legend()
# plt.savefig("result_log/af_avg.png", bbox_inches='tight', dpi=200)
# plt.show()

NameError: name 'concatenated_df' is not defined